In [1]:
#imports
import os
import pandas as pd

WORK WITH CITIES

In [2]:
#get city names
cities_csv_file = os.path.join('historical-hourly-weather-data/city_attributes.csv')

In [3]:
#read file
cities_df = pd.read_csv(cities_csv_file)

In [4]:
#only interested in city and country names
cities_df = cities_df[['City','Country']]

In [5]:
#only need US cities
cities_df = cities_df.loc[cities_df['Country'] == 'United States']

In [6]:
#add states to these cities
cities_df['State'] = ['Oregon', 'California', 'Washington', 'California', 'California', 'Nevada', 'Arizona', 'New Mexico', 'Colorado', 'Texas', 'Texas', 'Texas', 'Missouri', 'Minnesota', 'Missouri', 'Illinois', 'Tennessee', 'Indiana', 'Georgia', 'Michigan', 'Florida', 'North Carolina', 'Florida', 'Pennsylvania', 'Pennsylvania', 'New York', 'Massachusetts']

In [7]:
#get codes
#read file
codes_xl = os.path.join('FRPP GLC United States.xlsx')
codes_df = pd.read_excel(codes_xl, header=1, usecols='B,D,E')

In [8]:
#get ready to merge
cities_df['City'] = cities_df['City'].str.upper()
cities_df['State'] = cities_df['State'].str.upper()
cities_df = cities_df[['City', 'State']]

In [9]:
#merge
merged_cities = pd.merge(codes_df, cities_df, left_on=['City Name', 'State Name'], right_on=['City', 'State'])
merged_cities = merged_cities.drop_duplicates()
merged_cities.reset_index(inplace=True)

#only need these columns
merged_cities = merged_cities[['City Code', 'City', 'State']]

WORKING WITH FATALITIES

In [10]:
#get the accident data
fatal_2016_csv = os.path.join('fatal_2016.csv')
fatal_2015_csv = os.path.join('fatal_2015.csv')
fatal_2015_df = pd.read_csv(fatal_2015_csv)
fatal_2016_df = pd.read_csv(fatal_2016_csv)

In [11]:
#get a list of the codes where accidents occurred in 2015 and 2016
fatal_2015_codes = [x for x in fatal_2015_df['city']]
fatal_2016_codes = [x for x in fatal_2016_df['city']]

In [12]:
#create one list and then flatten the list of lists
all_codes = []
all_codes.append(fatal_2015_codes)
all_codes.append(fatal_2016_codes)
flat_codes = [y for x in all_codes for y in x]

In [13]:
#remove the duplicates
flat_codes = list(set(flat_codes))

In [14]:
#remove the codes in merged_cities that didn't have a fatal accident occur in them
merged_cities = merged_cities.loc[merged_cities['City Code'].isin(flat_codes)]

In [15]:
#get list of codes
codes_list = list(merged_cities['City Code'])

In [16]:
#select occurrences that occurred in the cities listed above
selected_2015 = fatal_2015_df.loc[fatal_2015_df['city'].isin(codes_list)]
selected_2016 = fatal_2016_df.loc[fatal_2016_df['city'].isin(codes_list)]
selected_fatal = selected_2015.append(selected_2016)

In [17]:
#df with all traffic fatalities that occurred in cities during 2015-2016, for which we have weather data
selected_fatal.reset_index(drop=True, inplace=True)

In [18]:
#row zero above is for North Bend, Oregon. City codes are re-used across states, but we only have the 26 cities of interest.
#we'll have to fix this like so:
merged_cities['State'] = merged_cities['State'].str.title()
trimmed_fatal = pd.merge(selected_fatal, merged_cities, left_on=['city', 'state_name'], right_on=['City Code', 'State'])
trimmed_fatal = trimmed_fatal[['City Code', 'year_of_crash', 'month_of_crash', 'day_of_crash', 'hour_of_crash', 'atmospheric_conditions_name', 'atmospheric_conditions_1_name', 'atmospheric_conditions_2_name', 'number_of_fatalities']]

In [19]:
#format: '10/1/2012 12:00'    
def datetime_maker(row):
    test_string = f"{row['month_of_crash']}/{row['day_of_crash']}/{row['year_of_crash']} {row['hour_of_crash']}:00"
    return test_string

In [20]:
# create series of datetime strings
trimmed_fatal['crash_time'] = trimmed_fatal.apply(datetime_maker, axis=1)

In [21]:
#select columns
trimmed_fatal = trimmed_fatal[['City Code', 'crash_time', 'atmospheric_conditions_name', 'atmospheric_conditions_1_name','atmospheric_conditions_2_name', 'number_of_fatalities']]

In [22]:
#comment to avoid recreation of file
#trimmed_fatal.to_csv(path_or_buf=os.path.join('fatalities.csv'), index=False)

In [23]:
#comment to avoid recreation of file
merged_cities['City'] = merged_cities['City'].str.title()
#merged_cities.to_csv(path_or_buf=os.path.join('location.csv'), index=False)

WORKING WITH WEATHER

In [51]:
#lists of city names and crash times that will be pulled from the weather csv files
city_name_list = list(merged_cities['City'])
crash_time_list = list(set(trimmed_fatal['crash_time'])) #only need unique values here...

In [52]:
#description, temperature, humidity, pressure, wind speed
base_path = os.path.join('historical-hourly-weather-data/')
#these lines are set up to ensure we keep the format: '01/01/2015 12:00'
my_dateparser = lambda x: pd.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

In [53]:
#get the weather data into dataframes
hums_df = pd.read_csv(filepath_or_buffer=(base_path + 'humidity.csv'), parse_dates=['datetime'], date_parser = my_dateparser)
descs_df = pd.read_csv(filepath_or_buffer=(base_path + 'weather_description.csv'), parse_dates=['datetime'], date_parser = my_dateparser)
temps_df = pd.read_csv(filepath_or_buffer=(base_path + 'temperature.csv'), parse_dates=['datetime'], date_parser = my_dateparser)
press_df = pd.read_csv(filepath_or_buffer=(base_path + 'pressure.csv'), parse_dates=['datetime'], date_parser = my_dateparser)
wind_df = pd.read_csv(filepath_or_buffer=(base_path + 'wind_speed.csv'), parse_dates=['datetime'], date_parser = my_dateparser)

In [54]:
#need to format the datetime column
def string_maker(date_time_object):
    test_string = date_time_object.datetime.strftime("%m/%d/%Y %H:%M")
    return test_string

In [55]:
#apply the function to format the dates
hums_df['datetime'] = hums_df.apply(string_maker, axis=1)
descs_df['datetime'] = descs_df.apply(string_maker, axis=1)
temps_df['datetime'] = temps_df.apply(string_maker, axis=1)
press_df['datetime'] = press_df.apply(string_maker, axis=1)
wind_df['datetime'] = wind_df.apply(string_maker, axis=1)

In [58]:
#filter out the crash times we have
hums_df = hums_df.loc[hums_df['datetime'].isin(crash_time_list)]
descs_df = descs_df.loc[descs_df['datetime'].isin(crash_time_list)]
temps_df = temps_df.loc[temps_df['datetime'].isin(crash_time_list)]
press_df = press_df.loc[press_df['datetime'].isin(crash_time_list)]
wind_df = wind_df.loc[wind_df['datetime'].isin(crash_time_list)]


,datetime,Vancouver,Portland,San Francisco,Seattle,Los Angeles,San Diego,Las Vegas,Phoenix,Albuquerque,...,Philadelphia,New York,Montreal,Boston,Beersheba,Tel Aviv District,Eilat,Haifa,Nahariyya,Jerusalem
26496,10/10/2015 12:00,3.0,2.0,3.0,5.0,1.0,2.0,1.0,0.0,1.0,...,1.0,2.0,3.0,3.0,1.0,7.0,7.0,4.0,4.0,6.0
26501,10/10/2015 17:00,8.0,6.0,3.0,2.0,1.0,2.0,1.0,1.0,1.0,...,1.0,2.0,3.0,8.0,2.0,5.0,8.0,5.0,5.0,3.0
26503,10/10/2015 19:00,4.0,5.0,5.0,7.0,1.0,1.0,3.0,4.0,2.0,...,1.0,2.0,5.0,6.0,2.0,6.0,5.0,4.0,4.0,3.0
26506,10/10/2015 22:00,9.0,9.0,7.0,12.0,6.0,2.0,4.0,3.0,2.0,...,1.0,2.0,6.0,7.0,2.0,1.0,2.0,4.0,4.0,1.0
26507,10/10/2015 23:00,6.0,3.0,8.0,6.0,4.0,5.0,5.0,2.0,3.0,...,1.0,2.0,6.0,6.0,1.0,1.0,2.0,5.0,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37249,12/31/2016 13:00,1.0,1.0,1.0,2.0,1.0,1.0,0.0,3.0,1.0,...,2.0,4.0,1.0,5.0,4.0,6.0,4.0,2.0,2.0,6.0
37252,12/31/2016 16:00,3.0,0.0,3.0,3.0,1.0,2.0,1.0,2.0,2.0,...,3.0,3.0,4.0,4.0,1.0,1.0,3.0,1.0,1.0,1.0
37254,12/31/2016 18:00,3.0,0.0,5.0,4.0,3.0,3.0,1.0,5.0,3.0,...,3.0,6.0,4.0,5.0,1.0,4.0,3.0,2.0,2.0,3.0
37257,12/31/2016 21:00,4.0,2.0,4.0,3.0,4.0,3.0,1.0,3.0,3.0,...,4.0,7.0,8.0,5.0,1.0,1.0,2.0,3.0,3.0,1.0
